# RAG 개발: PDF 청킹, Qdrant 적재, Azure OpenAI 검색
이 노트북은 PDF 파일을 청킹하여 Qdrant에 임베딩을 저장하고, Azure OpenAI를 활용해 검색하는 전체 과정을 다룹니다.

## 절차
1. 필수 라이브러리 설치
2. 환경 변수 및 라이브러리 임포트
3. PDF 파일 로드 및 청킹
4. Azure OpenAI 임베딩 객체 생성
5. Qdrant에 임베딩 및 문서 저장
6. Qdrant에서 유사도 검색


In [1]:
# 1. 필수 라이브러리 설치
!pip install PyPDF2 pypdf qdrant-client requests tqdm langchain-community langchain-openai python-dotenv

In [2]:
# 2. 환경 변수 및 라이브러리 임포트
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import Qdrant

# .env 파일 로드 및 환경 변수 설정
load_dotenv("../.env")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
QDRANT_URL = os.getenv("QDRANT_URL")
PDF_PATH = "../upload/folder/NOSS - P853.002.4.2021 - TVET Instuction - Complete.pdf"

print(f"QDRANT_URL: {QDRANT_URL}")
if QDRANT_URL and ("qdrant:6333" in QDRANT_URL):
    print("경고: QDRANT_URL이 'qdrant:6333'으로 되어 있습니다. 노트북에서 Qdrant에 접근하려면 반드시 'http://localhost:6333'으로 설정해야 합니다! .env 파일을 수정하세요.")

QDRANT_URL: http://localhost:6333


## PDF 파일 로드 및 청킹
PDF 파일을 불러와서 텍스트를 일정 길이로 청킹합니다.

In [3]:
# 3. PDF 파일 로드 및 텍스트 청킹
# PDF 파일을 LangChain의 PyPDFLoader로 불러오고, RecursiveCharacterTextSplitter로 텍스트를 청킹합니다.
loader = PyPDFLoader(PDF_PATH)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(f"총 청크 개수: {len(docs)}")
print(f"첫 번째 청크 예시:\n{docs[0].page_content[:300]}")

총 청크 개수: 388
첫 번째 청크 예시:
Jabatan Pembangunan Kemahiran 
Kementerian Sumber Manusia, Malaysia 
 
 
NATIONAL OCCUPATIONAL SKILLS STANDARD 
(STANDARD KEMAHIRAN PEKERJAAN KEBANGSAAN) 
 
 
 
 
 
P853-002-4:2021 
 
TVET INSTRUCTION  
 
PENGAJARAN TVET 
 
LEVEL 4


## Azure OpenAI 임베딩 생성
각 청크에 대해 Azure OpenAI의 text-embedding-ada-002 모델을 사용해 임베딩을 생성합니다.

In [ ]:
# 필요한 라이브러리 임포트
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
# from langchain.embeddings import AzureOpenAIEmbeddings
# from langchain.vectorstores import Qdrant
from tqdm import tqdm

# Azure OpenAI 임베딩 생성 함수 (직접 API 호출 - 선택사항)
def get_embedding_azure_openai(text, endpoint, api_key, deployment):
    """
    Azure OpenAI API를 직접 호출하여 임베딩 생성
    (LangChain 사용시에는 불필요)
    """
    url = f"{endpoint}/openai/deployments/{deployment}/embeddings?api-version=2025-01-01-preview"
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key
    }
    data = {
        "input": text,
        "model": deployment
    }
    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()
    return response.json()["data"][0]["embedding"]

# Qdrant URL 점검 및 안내
print(f"QDRANT_URL: {QDRANT_URL}")
if QDRANT_URL and ("qdrant:6333" in QDRANT_URL):
    print("경고: QDRANT_URL이 'qdrant:6333'으로 되어 있습니다. 노트북에서 Qdrant에 접근하려면 반드시 'http://localhost:6333'으로 설정해야 합니다! .env 파일을 수정하세요.")

# 4. Azure OpenAI 임베딩 객체 생성
# LangChain의 AzureOpenAIEmbeddings를 사용해 임베딩 객체를 생성합니다.
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    azure_deployment=AZURE_OPENAI_EMBEDDING_DEPLOYMENT,
    api_version="2025-01-01-preview",
    chunk_size=1000
)

# docs 변수가 미리 정의되어 있어야 함
# 예: docs = [Document(page_content="...", metadata={...}), ...]

# Qdrant VectorStore에 임베딩 및 문서 저장
if 'docs' in locals() and docs:  # docs 변수 존재 확인
    qdrant = Qdrant.from_documents(
        docs,
        embeddings,
        url=QDRANT_URL,
        collection_name="rag_chunks"
    )
    print(f"Qdrant에 {len(docs)}개 청크 임베딩 저장 완료.")
else:
    print("오류: 'docs' 변수가 정의되지 않았습니다.")

https://hspar-m7k2pfor-swedencentral.openai.azure.com/
FYKi43LLv1e3BWFkQpKT4QiTc7dzbhkZ0r1kV3CimDz8iRDWy854JQQJ99BBACfhMk5XJ3w3AAAAACOGRdJz
text-embedding-ada-002
QDRANT_URL: http://localhost:6333
Qdrant에 388개 청크 임베딩 저장 완료.
Qdrant에 388개 청크 임베딩 저장 완료.


## Qdrant에 임베딩 저장
생성된 임베딩을 Qdrant에 저장합니다. 컬렉션이 없으면 생성합니다.

In [5]:
# docs에서 텍스트만 추출하여 chunks 리스트 생성
chunks = [doc.page_content for doc in docs]

# Qdrant에 임베딩 저장 함수
def upsert_embeddings_to_qdrant(embeddings, chunks, qdrant_url, collection_name="rag_chunks", vector_size=1536):
    client = QdrantClient(url=qdrant_url)
    # 컬렉션 생성 (존재하지 않으면)
    if collection_name not in [c.name for c in client.get_collections().collections]:
        client.recreate_collection(
            collection_name=collection_name,
            vectors_config=models.VectorParams(size=vector_size, distance=models.Distance.COSINE)
        )
    # 포인트 업서트
    points = []
    for i, (emb, chunk) in enumerate(zip(embeddings, chunks)):
        points.append(models.PointStruct(
            id=str(uuid.uuid4()),
            vector=emb,
            payload={"text": chunk, "chunk_id": i}
        ))
    client.upsert(collection_name=collection_name, points=points)
    print(f"Qdrant에 {len(points)}개 임베딩 저장 완료.")

# 5. Qdrant에 임베딩 및 문서 저장
# LangChain Qdrant VectorStore를 사용해 임베딩과 문서를 Qdrant에 저장합니다.
if docs:
    qdrant = Qdrant.from_documents(
        docs,
        embeddings,
        url=QDRANT_URL,
        collection_name="rag_chunks"
    )
    print(f"Qdrant에 {len(docs)}개 청크 임베딩 저장 완료.")
else:
    print("오류: docs 리스트가 비어 있습니다.")

Qdrant에 388개 청크 임베딩 저장 완료.


## Qdrant 검색 및 결과 확인
사용자 쿼리를 임베딩하여 Qdrant에서 유사한 청크를 검색합니다.

In [6]:
# 6. Qdrant에서 유사도 검색
# 사용자가 입력한 쿼리를 임베딩하여 Qdrant에서 유사한 청크를 검색합니다.
user_query = "이 문서의 주요 목적은 무엇인가요?"
if 'qdrant' in locals():
    results = qdrant.similarity_search(user_query, k=5)
    for i, doc in enumerate(results, 1):
        print(f"[{i}] {doc.page_content[:300]}")
        print("-"*60)
else:
    print("Qdrant 인스턴스가 없습니다. 이전 셀을 먼저 실행하세요.")

[1] TABLE OF CONTENTS 
Preface ....................................................................................................................................... i 
Abbreviation .........................................................................................................................
------------------------------------------------------------
[2] TABLE OF CONTENTS 
Preface ....................................................................................................................................... i 
Abbreviation .........................................................................................................................
------------------------------------------------------------
[3] TABLE OF CONTENTS 
Preface ....................................................................................................................................... i 
Abbreviation .....................................................................................